In [5]:
from openclean.data.load import dataset
from openclean.op.constraint import EqIgnoreCase, IsEmpty
from openclean.operator.transform.filter import Filter, Ignore

In [6]:
ds = dataset('../data/DOB_Permit_Issuance.tsv')

In [7]:
brooklyn = Filter(EqIgnoreCase('borough', 'Brooklyn')).apply(ds)

TypeError: apply() missing 1 required positional argument: 'state'

In [8]:
brooklyn.head()

NameError: name 'brooklyn' is not defined

In [3]:
from openclean.op.group.fd import FDViolation

In [4]:
violations = FDViolation(lhs=['borough', 'block'], rhs='zip_code').apply(ds)

NameError: name 'ds' is not defined

In [1]:
for key, f in violations:
    print(key)
    print(f['zip_code'])

NameError: name 'violations' is not defined

In [ ]:
def has_outlier(df, state):
    return len(state.outliers) == 0

def get_outlier(df, state):
    return state.prev.outliers[0]


ForEach([
    HighFrequencyOutlier('zip_code', threshold=0.75),
    If(HasOutlier(), [
        If(Confirm(message='Replace all values with most frequent one'), [
            ReplaceAll('zip_code', GetOutlier())
        ])
    ])
]).apply(violations)

In [ ]:
ds = dataset('../data/DOB_Permit_Issuance.tsv')

pipeline = [
    Ignore(IsEmpty('borough')),
    FDViolation(lhs=['borough', 'block'], rhs='zip_code'),
    ForEach([
        HighFrequencyOutlier('zip_code', threshold=0.75),
        If(HasOutlier(), [
            If(Confirm(message='Replace all values with most frequent one'), [
                ReplaceAll('zip_code', GetOutlier())
            ])
        ])
    ]),
    Union(),
    Merge(ds)
]

pipeline.apply(ds)

In [ ]:
from openclean.op import Ignore, FDViolation, ...

ds = dataset('../data/DOB_Permit_Issuance.tsv')

non_empty = Ignore(ds, IsEmpty('borough'))  # Or Ignore(IsEmpty('borough')).apply(ds)
violations = FDViolation(non_empty, lhs=['borough', 'block'], rhs='zip_code')

result = pd.DataFrame(columns=ds.columns)
for key, df in violations:
    outlier = HighFrequencyOutlier(df, 'zip_code', threshold=0.75)
    if outlier.size == 1:
        if Confirm(df, message='Replace all values with most frequent one'):
            result = Union(result, ReplaceAll(df, 'zip_code', outlier.iloc[0,0]))
ds = Merge(ds, result)

In [ ]:
@vizier.command(name='FDRepair')
@vizier.argument(name='ds', type='dataset')
@vizier.argument(name='lhs', type='collist')
@vizier.argument(name='rhs', type='collist')
def FDMajorityRepair(ds, lhs, rhs):
    violations = FDViolation(non_empty, lhs=lhs, rhs=rhs)
    result = pd.DataFrame(columns=ds.columns)
    for key, df in violations:
        outlier = HighFrequencyOutlier(df, rhs, threshold=0.75)
        if outlier.size == 1:
            if Confirm(df, message='Replace all values with most frequent one'):
                result = Union(result, ReplaceAll(df, rhs, outlier.iloc[0]))
    return result


if __name__ == '__main__':
    ds = dataset('../data/DOB_Permit_Issuance.tsv')
    non_empty = Ignore(ds, IsEmpty('borough'))
    repair = FDMajorityRepair(non_empty, ['borough', 'block'], 'zip_code')
    ds = Merge(ds, repair)